In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

from sklearn.metrics import silhouette_score

from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error, explained_variance_score



from sklearn.model_selection import train_test_split


pd.options.display.max_columns = None
pd.options.display.float_format = "{:,.4f}".format



In [2]:
#df = pd.read_csv('Data_Set.csv', low_memory=False, index_col='custid')


df = pd.read_csv('Data_Set.csv', low_memory=False)
#df.index = df['custid']

#df_cluster = pd.read_csv('clusters.csv')
#df = pd.concat([df, df_cluster], axis=1)

In [3]:
df.loc[:, ['total_spent']] = df['cardspent'] + df['card2spent']
df.loc[:, ['debtinc']] =  df['debtinc']*df['income']/100.0
df.drop(['cardspent', 'card2spent'], axis=1, inplace=True)

#df.drop(columns=['carvalue'], inplace=True)


In [4]:
X = df.drop(columns=['total_spent']).copy()
y = df['total_spent'].copy()
X.replace('#NULL!', np.nan, inplace=True) 
X.fillna(X.mean(), inplace=True)
X.fillna(0, inplace=True)
#X.loc[:, ['region', 'townsize', 'gender', 'age', 'agecat']] = X[['region', 'townsize', 'gender', 'age', 'agecat']].astype(np.float64).astype(np.int64)
X = X.convert_dtypes()
cols = X.select_dtypes(float).columns
X[cols] = X[cols].astype(np.float64)
cols = X.select_dtypes(int).columns
X[cols] = X[cols].astype(np.int64)
cols = X.select_dtypes('object').columns
X[cols] = X[cols].astype(np.float64)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_temp = X_train.copy()
X_test_temp = X_test.copy()
X_train_temp.drop(columns=['custid', 'birthmonth'], inplace=True)
X_test_temp.drop(columns=['custid', 'birthmonth'], inplace=True)

In [6]:
df_mean = None
#ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe = OneHotEncoder(sparse=False, drop='first')
std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
transformer = PowerTransformer()

scaler_y = StandardScaler()
transformer_y = PowerTransformer()

In [7]:
def calculate_cardmon_avg(df, is_test=False):
    global ohe, transformer, std_scaler, minmax_scaler, df_mean
    
    log_columns_original = list(filter(lambda x: x.startswith('ln'), df.columns))
    df.drop(log_columns_original, axis=1, inplace=True)
    
    df.replace('#NULL!', np.nan, inplace=True)   

    df.drop(columns=['birthmonth'], inplace=True)
    df.drop(columns=['custid'], inplace=True)
    
    df.loc[:, ['cardmon_total']] = df[['cardmon', 'wiremon', 'longmon', 'equipmon', 'tollmon']].sum(axis=1) 
    df.loc[:, ['cardten_total']] = df[['cardten', 'wireten', 'longten', 'equipten', 'tollten']].sum(axis=1)#/(df['tenure']*1.0)
    
    if not(is_test):
        df_mean = df.mean()
        
    df.fillna(df_mean, inplace=True)
    df.fillna(0, inplace=True)
    
#    df.loc[:, 'have_pets'] = df['pets'].apply(lambda x: 1 if x > 0 else 0)
    df.loc[:, ['have_car']] = df[['carbought', 'carbuy']].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
    
    features_own_tv = list(filter(lambda x: x.lower().find('own')==0, df.columns))
    df.loc[:, 'own_tv_set_1'] = df[features_own_tv[:4]].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
    df.loc[:, 'own_tv_set_2'] = df[features_own_tv[4:]].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

    
    

    

#    df.loc[:, 'hourstv_cat'] = pd.cut((df['hourstv']/7.0), bins=[-1, 2, 3.5, 6]).apply(lambda x:x.mid.astype(np.float64))
    df.loc[:, ['debt_total']] = df[['creddebt', 'othdebt']].sum(axis=1)
#    df.loc[:, ['cardtype_cat']] = df[['cardtype', 'card2type']].apply(lambda x:max(x[0], x[1]), axis=1)

#    df.loc[:, ['response']] = df[['response_01', 'response_02', 'response_03']].sum(axis=1).apply(lambda x: 1 if x>0 else 0)

     
    features_pets = [x for x in df.columns if x.lower().startswith('pets') and x.lower()!='pets']
    df.drop(features_pets, axis=1, inplace=True)
    
    
    df.drop(['cardten', 'wireten', 'longten', 'equipten', 'tollten'], axis=1, inplace=True)
    df.drop(['cardmon', 'wiremon', 'longmon', 'equipmon', 'tollmon'], axis=1, inplace=True)
    df.drop(['callcard', 'wireless', 'equip', 'tollfree'], axis=1, inplace=True)
    
    df.drop(columns=['region', 'townsize', 'gender', 'ed', 'spoused', 'address'], inplace=True)
    df.drop(columns=['polview', 'polparty', 'polcontrib', 'vote',
                     'cardbenefit', 'card2benefit', 'churn', 'multline', 'voice', 'news'
                    , 'response_01', 'response_02', 'response_03'], inplace=True)
    df.drop(columns=['pager'], inplace=True)
    df.drop(columns=['active', 'bfast', 'internet', 
    'callid',
    'callwait',
    'forward',
    'confer',
    'ebill'], inplace=True)

    df.drop(columns=['tenure'], inplace=True)
#    df.drop(columns=['card', 'card2'], inplace=True)
#    df.drop(columns=['cardtype', 'card2type'], inplace=True)
    df.drop(['spousedcat', 'cars'], axis=1, inplace=True)
    df.drop(['addresscat', 'hometype'], axis=1, inplace=True)
    df.drop(['inccat'], axis=1, inplace=True)
    df.drop(columns=['union', 'retire', 'cardfee', 'card2fee', 'carown'], inplace=True)
    
    commute_to_drop = [x for x in df.columns if 'commute' in x.lower() and x.lower() not in ['commutecat', 'commutetime']]
    df.drop(columns=commute_to_drop, inplace=True)
    
    df.drop(columns=['carbought', 'carbuy'], inplace=True)
    
    df.drop(['default', 'marital'], axis=1, inplace=True)
    df.drop(features_own_tv, axis=1, inplace=True)
#    df.drop(['own_tv_set_1'], axis=1, inplace=True)
#    df.drop(columns=['hourstv'], inplace=True)

    df.drop(columns=['othdebt'], inplace=True)
    df.drop(columns=['creddebt'], inplace=True)
    
#    df.drop(columns=['reason'], inplace=True)    
    df.drop(['carvalue'], axis=1, inplace=True)
    df.drop(['age'], axis=1, inplace=True)
    
    df.loc[:, ['cardtenure_cat']] = df[['cardtenure', 'card2tenure']].sum(axis=1)
#    df.loc[:, ['cardtenurecat_cat']] = df[['cardtenurecat', 'card2tenurecat']].sum(axis=1)
    
#    df.drop(columns=['cardtenure', 'card2tenure', 'cardtenurecat', 'card2tenurecat'], inplace=True)
    
    '''

    df.drop(columns=['cartype', 'cardtenure', 'card2tenure','employ',
    'cardtenurecat', 'card2tenurecat'], inplace=True)




    


    
    df.loc[:, ['tenure_cat']] = np.max(df[['cardtenurecat', 'card2tenurecat']], axis=1)
    
#    df.drop(columns=['income'], inplace=True)
    

    


    




#    df.drop(columns=['debtinc'], inplace=True)


    
    df.drop(columns=['tenure_cat'], inplace=True)
#    df.drop(columns=['cardtype_cat'], inplace=True)
    '''    
    

    features_categorical = [f for f in df.columns if df[f].nunique()<=60]
    features_continuous = list(set(df.columns) - set(features_categorical))
    df.loc[:, features_categorical] = df[features_categorical].astype(np.int64)
    temp = df[features_categorical].astype(np.int64)
    
    if not(is_test):
        ohe.fit(temp)
    col_names = ohe.get_feature_names(temp.columns)
    df_ohe = pd.DataFrame(ohe.transform(temp), columns=col_names, index=df.index)
    
    df_scaled = df[features_continuous]

    '''   
    if not(is_test):
        transformer.fit(std_scaler.fit_transform(df[features_continuous]))
    df_scaled = pd.DataFrame(transformer.transform(std_scaler.transform(df_scaled)),
                             columns=features_continuous, index=df.index)
    '''
    '''    
    if not(is_test):
        transformer.fit(df_scaled)
    df_scaled = pd.DataFrame(transformer.transform(df_scaled),
                             columns=features_continuous, index=df.index)
   ''' 

    df_scaled = pd.DataFrame(
        np.log1p(df_scaled), columns=features_continuous, index=df.index)
    
    df_input = pd.concat([df_scaled, df_ohe], axis=1)
    df_input.loc[:, :] = df_input.convert_dtypes()
    cols = df_input.select_dtypes(float).columns
    df_input[cols] = df_input[cols].astype(np.float64)
    cols = df_input.select_dtypes(int).columns
    df_input[cols] = df_input[cols].astype(np.int64)

    return df_input

    
X_train = calculate_cardmon_avg(X_train.copy())
X_test = calculate_cardmon_avg(X_test.copy(), is_test=True)



In [8]:
X_train.shape, X_test.shape

((4000, 286), (1000, 286))

In [9]:
X_train

,cardmon_total,cardtenure_cat,debt_total,income,cardten_total,debtinc,agecat_3,agecat_4,agecat_5,agecat_6,edcat_2,edcat_3,edcat_4,edcat_5,jobcat_2,jobcat_3,jobcat_4,jobcat_5,jobcat_6,employ_1,employ_2,employ_3,employ_4,employ_5,employ_6,employ_7,employ_8,employ_9,employ_10,employ_11,employ_12,employ_13,employ_14,employ_15,employ_16,employ_17,employ_18,employ_19,employ_20,employ_21,employ_22,employ_23,employ_24,employ_25,employ_26,employ_27,employ_28,employ_29,employ_30,employ_31,employ_32,employ_33,employ_34,employ_35,employ_36,employ_37,employ_38,employ_39,employ_40,employ_41,employ_42,employ_43,employ_44,employ_45,employ_46,employ_47,employ_48,employ_49,employ_51,employ_52,empcat_2,empcat_3,empcat_4,empcat_5,jobsat_2,jobsat_3,jobsat_4,jobsat_5,reside_2,reside_3,reside_4,reside_5,reside_6,reside_7,reside_8,reside_9,pets_1,pets_2,pets_3,pets_4,pets_5,pets_6,pets_7,pets_8,pets_9,pets_10,pets_11,pets_12,pets_13,pets_14,pets_15,pets_16,pets_18,pets_19,pets_21,homeown_1,cartype_0,cartype_1,carcatvalue_1,carcatvalue_2,carcatvalue_3,commutecat_2,commutecat_3,commutecat_4,commutecat_5,commutetime_8,commutetime_9,commutetime_10,commutetime_11,commutetime_12,commutetime_13,commutetime_14,commutetime_15,commutetime_16,commutetime_17,commutetime_18,commutetime_19,commutetime_20,commutetime_21,commutetime_22,commutetime_23,commutetime_24,commutetime_25,commutetime_26,commutetime_27,commutetime_28,commutetime_29,commutetime_30,commutetime_31,commutetime_32,commutetime_33,commutetime_34,commutetime_35,commutetime_36,commutetime_37,commutetime_38,commutetime_39,commutetime_40,commutetime_41,commutetime_42,commutetime_43,commutetime_44,commutetime_45,commutetime_46,commutetime_47,commutetime_48,reason_2,reason_3,reason_4,reason_9,card_2,card_3,card_4,card_5,cardtype_2,cardtype_3,cardtype_4,cardtenure_1,cardtenure_2,cardtenure_3,cardtenure_4,cardtenure_5,cardtenure_6,cardtenure_7,cardtenure_8,cardtenure_9,cardtenure_10,cardtenure_11,cardtenure_12,cardtenure_13,cardtenure_14,cardtenure_15,cardtenure_16,cardtenure_17,cardtenure_18,cardtenure_19,cardtenure_20,cardtenure_21,cardtenure_22,cardtenure_23,cardtenure_24,cardtenure_25,cardtenure_26,cardtenure_27,cardtenure_28,cardtenure_29,cardtenure_30,cardtenure_31,cardtenure_32,cardtenure_33,cardtenure_34,cardtenure_35,cardtenure_36,cardtenure_37,cardtenure_38,cardtenure_39,cardtenure_40,cardtenurecat_2,cardtenurecat_3,cardtenurecat_4,cardtenurecat_5,card2_2,card2_3,card2_4,card2_5,card2type_2,card2type_3,card2type_4,card2tenure_1,card2tenure_2,card2tenure_3,card2tenure_4,card2tenure_5,card2tenure_6,card2tenure_7,card2tenure_8,card2tenure_9,card2tenure_10,card2tenure_11,card2tenure_12,card2tenure_13,card2tenure_14,card2tenure_15,card2tenure_16,card2tenure_17,card2tenure_18,card2tenure_19,card2tenure_20,card2tenure_21,card2tenure_22,card2tenure_23,card2tenure_24,card2tenure_25,card2tenure_26,card2tenure_27,card2tenure_28,card2tenure_29,card2tenure_30,card2tenurecat_2,card2tenurecat_3,card2tenurecat_4,card2tenurecat_5,hourstv_6,hourstv_7,hourstv_8,hourstv_9,hourstv_10,hourstv_11,hourstv_12,hourstv_13,hourstv_14,hourstv_15,hourstv_16,hourstv_17,hourstv_18,hourstv_19,hourstv_20,hourstv_21,hourstv_22,hourstv_23,hourstv_24,hourstv_25,hourstv_26,hourstv_27,hourstv_28,hourstv_29,hourstv_30,hourstv_31,hourstv_32,hourstv_33,hourstv_34,hourstv_35,hourstv_36,have_car_1,own_tv_set_1_1,own_tv_set_2_1
4227,3.4420,3.6636,2.0869,3.8286,7.0803,2.0875,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4676,5.0854,4.1589

In [10]:
#y_train = transformer_y.fit_transform(y_train.values.reshape(-1, 1))
#y_test = transformer_y.transform(y_test.values.reshape(-1, 1))

#y_train = transformer_y.fit_transform(scaler_y.fit_transform(y_train.values.reshape(-1, 1)))
#y_test = transformer_y.transform(scaler_y.transform(y_test.values.reshape(-1, 1)))

y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

In [11]:
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

svr_model = SVR()
linear_svr_model = LinearSVR(random_state=42)
lr_model = LinearRegression()
xgb_model = XGBRegressor(learning_rate=0.1, n_estimators=1000, random_state=42)
ridge_model = Ridge(random_state = 42)
lasso_model = Lasso(random_state = 42)
knn_model = KNeighborsRegressor(5,weights='uniform')
bay_model = BayesianRidge()
dec_model = DecisionTreeRegressor(random_state = 42)
rf_model = RandomForestRegressor(n_estimators = 300, random_state = 42)


model_list = [lr_model, bay_model, ridge_model, lasso_model, linear_svr_model, svr_model, xgb_model]

#model_list=[lr_model]

for model in model_list:
    print(str(model).split('(')[0])
    model.fit(X_train.values, y_train.ravel())


LinearRegression
BayesianRidge
Ridge
Lasso
LinearSVR


/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


SVR
XGBRegressor


In [12]:
X_train.columns[X_train.isna().any()]

Index([], dtype='object')

In [13]:
def check_model_accuracy(model_list, X_train, X_test, y_train, y_test):
    df_result = pd.DataFrame(columns=['r2_score train',
                                      'explained variance train',
                                      'rmse train',
                                      'mae train',
                                      'med absolute error train',
                                      'r2_score test',
                                      'explained variance test',                                      
                                      'rmse test',
                                      'mae test',
                                      'med absolute error test'
                                     ] )
    
    for model in model_list:
        model_name = str(model).split('(')[0]

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        df_result.loc[model_name] = np.array([r2_score(y_train, y_train_pred),
                                              explained_variance_score(y_train, y_train_pred),
                                              mean_squared_error(y_train, y_train_pred, squared=True), 
                                              mean_absolute_error(y_train, y_train_pred), 
                                              median_absolute_error(y_train, y_train_pred), 
                                              r2_score(y_test, y_test_pred),
                                              explained_variance_score(y_test, y_test_pred),
                                              mean_squared_error(y_test, y_test_pred, squared=True), 
                                              median_absolute_error(y_test, y_test_pred),
                                              mean_absolute_error(y_test, y_test_pred)
                                             ])
    return df_result

In [14]:
check_model_accuracy(model_list, X_train, X_test, y_train, y_test)

,r2_score train,explained variance train,rmse train,mae train,med absolute error train,r2_score test,explained variance test,rmse test,mae test,med absolute error test
LinearRegression,0.3825,0.3825,0.2732,0.4027,0.3333,0.2835,0.2835,0.3320,0.3585,0.4513
BayesianRidge,0.3689,0.3689,0.2792,0.4049,0.3291,0.3003,0.3003,0.3242,0.3572,0.4433
Ridge,0.3818,0.3818,0.2735,0.4032,0.3341,0.2875,0.2876,0.3301,0.3612,0.4497
Lasso,0.0000,0.0000,0.4425,0.5029,0.3951,-0.0001,0.0000,0.4634,0.4205,0.5304
LinearSVR,0.3536,0.3538,0.2860,0.3956,0.3139,0.2525,0.2525,0.3464,0.3831,0.4587
SVR,0.3921,0.3940,0.2690,0.3788,0.2781,0.2580,0.2589,0.3438,0.3611,0.4536
XGBRegressor,0.9938,0.9938,0.0027,0.0387,0.0280,0.1625,0.1625,0.3880,0.3996,0.4876


In [15]:
sorted(list(X_train.columns))

['agecat_3',
 'agecat_4',
 'agecat_5',
 'agecat_6',
 'carcatvalue_1',
 'carcatvalue_2',
 'carcatvalue_3',
 'card2_2',
 'card2_3',
 'card2_4',
 'card2_5',
 'card2tenure_1',
 'card2tenure_10',
 'card2tenure_11',
 'card2tenure_12',
 'card2tenure_13',
 'card2tenure_14',
 'card2tenure_15',
 'card2tenure_16',
 'card2tenure_17',
 'card2tenure_18',
 'card2tenure_19',
 'card2tenure_2',
 'card2tenure_20',
 'card2tenure_21',
 'card2tenure_22',
 'card2tenure_23',
 'card2tenure_24',
 'card2tenure_25',
 'card2tenure_26',
 'card2tenure_27',
 'card2tenure_28',
 'card2tenure_29',
 'card2tenure_3',
 'card2tenure_30',
 'card2tenure_4',
 'card2tenure_5',
 'card2tenure_6',
 'card2tenure_7',
 'card2tenure_8',
 'card2tenure_9',
 'card2tenurecat_2',
 'card2tenurecat_3',
 'card2tenurecat_4',
 'card2tenurecat_5',
 'card2type_2',
 'card2type_3',
 'card2type_4',
 'card_2',
 'card_3',
 'card_4',
 'card_5',
 'cardmon_total',
 'cardten_total',
 'cardtenure_1',
 'cardtenure_10',
 'cardtenure_11',
 'cardtenure_12',
 '

In [16]:
sorted([f for f in X_train.columns if X_train[f].nunique()>60])

['cardmon_total',
 'cardten_total',
 'cardtenure_cat',
 'debt_total',
 'debtinc',
 'income']

In [17]:
lr_model.predict(X_train.iloc[:1, :]), y_train[1]

(array([5.68867003]), 4.363608049149864)

In [18]:
#act=transformer_y.inverse_transform(y_train[:1])
#pred=transformer_y.inverse_transform(lr_model.predict(X_train.iloc[:1, :]).reshape(-1, 1))

act=np.expm1(y_train[:2]).values
pred=np.expm1(lr_model.predict(X_train.iloc[:2, :]))



pred, act

(array([294.50122452, 347.08393958]), array([460.7 , 538.23]))

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [20]:
def create_deep_model(X, y):
#    act_f = tf.keras.layers.LeakyReLU(alpha=0.01)
    act_f = tf.nn.relu
    input_layer = keras.Input(shape=(X.shape[1],), name='input_layer')

    dense_layer = layers.Dense(32, kernel_initializer='normal',activation=act_f, name='dense_layer_1')(input_layer)
    dense_layer = layers.Dense(64, kernel_initializer='normal',activation=act_f, name='dense_layer_2')(dense_layer)
    dense_layer = layers.Dense(128, kernel_initializer='normal',activation=act_f, name='dense_layer_3')(dense_layer)
    dense_layer = layers.Dense(256, kernel_initializer='normal',activation=act_f, name='dense_layer_4')(dense_layer)
    dense_layer = layers.Dense(512, kernel_initializer='normal',activation=act_f, name='dense_layer_5')(dense_layer)


    output_layer = layers.Dense(1, name='output_layer')(dense_layer)

    model = keras.Model( inputs = [input_layer], 
                           outputs = [output_layer], name="reg_model")

    #optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer = tf.keras.optimizers.Adam(0.01)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    
    stop_callback = keras.callbacks.EarlyStopping(monitor='mae', mode='min', patience=10)
    model.fit(X, y, batch_size=16, epochs=1000, validation_split=0.2,
          callbacks=[stop_callback], verbose=1)
    return model




In [21]:
model = create_deep_model(X_train, y_train)


Epoch 1/1000
200/200 [==============================] - 1s 3ms/step - loss: 1.0011 - mae: 0.6659 - mse: 1.0011 - val_loss: 0.3942 - val_mae: 0.4911 - val_mse: 0.3942
Epoch 2/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.4137 - mae: 0.4951 - mse: 0.4137 - val_loss: 0.3762 - val_mae: 0.4602 - val_mse: 0.3762
Epoch 3/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.4569 - mae: 0.5215 - mse: 0.4569 - val_loss: 0.6040 - val_mae: 0.6557 - val_mse: 0.6040
Epoch 4/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.3750 - mae: 0.4721 - mse: 0.3750 - val_loss: 0.3422 - val_mae: 0.4493 - val_mse: 0.3422
Epoch 5/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.3465 - mae: 0.4551 - mse: 0.3465 - val_loss: 0.3297 - val_mae: 0.4337 - val_mse: 0.3297
Epoch 6/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.3475 - mae: 0.4562 - mse: 0.3475 - val_loss: 0.3374 - val_mae: 0.4373 - val_mse: 0.3374
Epoc

200/200 [==============================] - 0s 2ms/step - loss: 0.1092 - mae: 0.2574 - mse: 0.1092 - val_loss: 0.5241 - val_mae: 0.5592 - val_mse: 0.5241
Epoch 51/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.1057 - mae: 0.2528 - mse: 0.1057 - val_loss: 0.4596 - val_mae: 0.5308 - val_mse: 0.4596
Epoch 52/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.1164 - mae: 0.2632 - mse: 0.1164 - val_loss: 0.4858 - val_mae: 0.5465 - val_mse: 0.4858
Epoch 53/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.1080 - mae: 0.2520 - mse: 0.1080 - val_loss: 0.5072 - val_mae: 0.5531 - val_mse: 0.5072
Epoch 54/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.1059 - mae: 0.2508 - mse: 0.1059 - val_loss: 0.4884 - val_mae: 0.5456 - val_mse: 0.4884
Epoch 55/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.0952 - mae: 0.2387 - mse: 0.0952 - val_loss: 0.5171 - val_mae: 0.5655 - val_mse: 0.5171
Epoch 56/100

In [22]:
check_model_accuracy([model], X_train, X_test, y_train, y_test)

,r2_score train,explained variance train,rmse train,mae train,med absolute error train,r2_score test,explained variance test,rmse test,mae test,med absolute error test
<tensorflow.python.keras.engine.functional.Functional object at 0x7fac1ba0f550>,0.6400,0.6638,0.1593,0.2790,0.1990,-0.2710,-0.2455,0.5889,0.4779,0.5963


In [23]:
act=np.expm1(y_train[:2]).values
pred=np.expm1(model.predict(X_train.iloc[:2, :]))



pred, act

(array([[389.4167 ],
        [389.62784]], dtype=float32),
 array([460.7 , 538.23]))

In [24]:
model = create_deep_model(X_train_temp, y_train)

Epoch 1/1000
200/200 [==============================] - 1s 3ms/step - loss: 85.4297 - mae: 2.0515 - mse: 85.4297 - val_loss: 0.7684 - val_mae: 0.7076 - val_mse: 0.7684
Epoch 2/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.7060 - mae: 0.6541 - mse: 0.7060 - val_loss: 1.0903 - val_mae: 0.8900 - val_mse: 1.0903
Epoch 3/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.5740 - mae: 0.5913 - mse: 0.5740 - val_loss: 0.4303 - val_mae: 0.4970 - val_mse: 0.4303
Epoch 4/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.5054 - mae: 0.5552 - mse: 0.5054 - val_loss: 0.4011 - val_mae: 0.4804 - val_mse: 0.4011
Epoch 5/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.4621 - mae: 0.5188 - mse: 0.4621 - val_loss: 0.4564 - val_mae: 0.5247 - val_mse: 0.4564
Epoch 6/1000
200/200 [==============================] - 0s 2ms/step - loss: 0.4714 - mae: 0.5268 - mse: 0.4714 - val_loss: 0.4107 - val_mae: 0.4890 - val_mse: 0.4107
Ep

In [25]:
check_model_accuracy([model], X_train_temp, X_test_temp, y_train, y_test)

,r2_score train,explained variance train,rmse train,mae train,med absolute error train,r2_score test,explained variance test,rmse test,mae test,med absolute error test
<tensorflow.python.keras.engine.functional.Functional object at 0x7fac131cc340>,-0.0000,0.0000,0.4425,0.5031,0.3964,-0.0004,-0.0001,0.4635,0.4233,0.5307


In [26]:
act=np.expm1(y_train[:20]).values
pred=np.expm1(model.predict(X_train_temp.iloc[:20, :]))



pred, act

(array([[402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17282],
        [402.17245],
        [402.17245]], dtype=float32),
 array([ 460.7 ,  538.23,  413.31,  171.02,  238.31,  379.72,  596.52,
         279.97,  110.23, 1323.48,  365.25,  568.16,  186.83,  446.69,
         362.32, 1619.9 ,  411.25,  310.5 ,  654.37,  413.91]))

In [27]:
X_train

,cardmon_total,cardtenure_cat,debt_total,income,cardten_total,debtinc,agecat_3,agecat_4,agecat_5,agecat_6,edcat_2,edcat_3,edcat_4,edcat_5,jobcat_2,jobcat_3,jobcat_4,jobcat_5,jobcat_6,employ_1,employ_2,employ_3,employ_4,employ_5,employ_6,employ_7,employ_8,employ_9,employ_10,employ_11,employ_12,employ_13,employ_14,employ_15,employ_16,employ_17,employ_18,employ_19,employ_20,employ_21,employ_22,employ_23,employ_24,employ_25,employ_26,employ_27,employ_28,employ_29,employ_30,employ_31,employ_32,employ_33,employ_34,employ_35,employ_36,employ_37,employ_38,employ_39,employ_40,employ_41,employ_42,employ_43,employ_44,employ_45,employ_46,employ_47,employ_48,employ_49,employ_51,employ_52,empcat_2,empcat_3,empcat_4,empcat_5,jobsat_2,jobsat_3,jobsat_4,jobsat_5,reside_2,reside_3,reside_4,reside_5,reside_6,reside_7,reside_8,reside_9,pets_1,pets_2,pets_3,pets_4,pets_5,pets_6,pets_7,pets_8,pets_9,pets_10,pets_11,pets_12,pets_13,pets_14,pets_15,pets_16,pets_18,pets_19,pets_21,homeown_1,cartype_0,cartype_1,carcatvalue_1,carcatvalue_2,carcatvalue_3,commutecat_2,commutecat_3,commutecat_4,commutecat_5,commutetime_8,commutetime_9,commutetime_10,commutetime_11,commutetime_12,commutetime_13,commutetime_14,commutetime_15,commutetime_16,commutetime_17,commutetime_18,commutetime_19,commutetime_20,commutetime_21,commutetime_22,commutetime_23,commutetime_24,commutetime_25,commutetime_26,commutetime_27,commutetime_28,commutetime_29,commutetime_30,commutetime_31,commutetime_32,commutetime_33,commutetime_34,commutetime_35,commutetime_36,commutetime_37,commutetime_38,commutetime_39,commutetime_40,commutetime_41,commutetime_42,commutetime_43,commutetime_44,commutetime_45,commutetime_46,commutetime_47,commutetime_48,reason_2,reason_3,reason_4,reason_9,card_2,card_3,card_4,card_5,cardtype_2,cardtype_3,cardtype_4,cardtenure_1,cardtenure_2,cardtenure_3,cardtenure_4,cardtenure_5,cardtenure_6,cardtenure_7,cardtenure_8,cardtenure_9,cardtenure_10,cardtenure_11,cardtenure_12,cardtenure_13,cardtenure_14,cardtenure_15,cardtenure_16,cardtenure_17,cardtenure_18,cardtenure_19,cardtenure_20,cardtenure_21,cardtenure_22,cardtenure_23,cardtenure_24,cardtenure_25,cardtenure_26,cardtenure_27,cardtenure_28,cardtenure_29,cardtenure_30,cardtenure_31,cardtenure_32,cardtenure_33,cardtenure_34,cardtenure_35,cardtenure_36,cardtenure_37,cardtenure_38,cardtenure_39,cardtenure_40,cardtenurecat_2,cardtenurecat_3,cardtenurecat_4,cardtenurecat_5,card2_2,card2_3,card2_4,card2_5,card2type_2,card2type_3,card2type_4,card2tenure_1,card2tenure_2,card2tenure_3,card2tenure_4,card2tenure_5,card2tenure_6,card2tenure_7,card2tenure_8,card2tenure_9,card2tenure_10,card2tenure_11,card2tenure_12,card2tenure_13,card2tenure_14,card2tenure_15,card2tenure_16,card2tenure_17,card2tenure_18,card2tenure_19,card2tenure_20,card2tenure_21,card2tenure_22,card2tenure_23,card2tenure_24,card2tenure_25,card2tenure_26,card2tenure_27,card2tenure_28,card2tenure_29,card2tenure_30,card2tenurecat_2,card2tenurecat_3,card2tenurecat_4,card2tenurecat_5,hourstv_6,hourstv_7,hourstv_8,hourstv_9,hourstv_10,hourstv_11,hourstv_12,hourstv_13,hourstv_14,hourstv_15,hourstv_16,hourstv_17,hourstv_18,hourstv_19,hourstv_20,hourstv_21,hourstv_22,hourstv_23,hourstv_24,hourstv_25,hourstv_26,hourstv_27,hourstv_28,hourstv_29,hourstv_30,hourstv_31,hourstv_32,hourstv_33,hourstv_34,hourstv_35,hourstv_36,have_car_1,own_tv_set_1_1,own_tv_set_2_1
4227,3.4420,3.6636,2.0869,3.8286,7.0803,2.0875,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4676,5.0854,4.1589